In [1]:
import torch.nn as nn
import torch
import math
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
p = torch.rand([32, 65440])
y = torch.rand([32,65440])

In [15]:
def focal_loss(p, y, gamma = 2, alpha=1):
    """
    It used to suppress the presence of a large number of negative prediction.
    It works on image level not batch level.
    For any example/image, it keeps all the positive predictions and
     cut the number of negative predictions to make sure the ratio
     between the negative examples and positive examples is no more
     the given ratio for an image.
    Args:
        loss (N, num_priors): the loss for each example.
        labels (N, num_priors): the labels.
        neg_pos_ratio:  the ratio between the negative examples and positive examples.
    """
    neg = 1 - p
    term1 = torch.pow(neg,gamma)
    term2 = y
    term3 = torch.log(p)
    
    return -term1 * term2 * term3

In [16]:
focal_loss(p,y, gamma=5)

tensor([[8.2312e-07, 1.0796e-04, 1.1222e+00,  ..., 2.5963e+00, 2.0566e-02,
         2.7834e-03],
        [1.7884e-02, 8.8541e-02, 2.2278e-02,  ..., 2.7356e+00, 6.2279e-03,
         1.2206e-05],
        [1.6881e+00, 1.0304e-06, 7.1640e-02,  ..., 4.8990e-04, 1.1023e+00,
         6.2865e-03],
        ...,
        [7.4628e-02, 6.2864e-11, 1.2448e+00,  ..., 5.6112e-05, 6.3003e-06,
         2.3727e-01],
        [9.0898e-02, 1.1591e-02, 3.2926e-01,  ..., 3.6088e-01, 1.4848e-03,
         6.8981e-03],
        [3.4742e-01, 1.3188e-01, 2.7378e-02,  ..., 9.4455e-02, 1.4322e-03,
         2.3788e-01]])

In [ ]:
def focal_loss(confs, y_pred, gamma=2.0):
    #y_pred (tensor of shape (batch size, num_boxes, num_classes)
    #confs: [32, 9, 65440]
    alpha = torch.ones([len(confs[1])])
    alpha[0] = 0.1
    
    eps = K.epsilon()
    y_pred = K.clip(y_pred, eps, 1.-eps)
    pt = tf.where(tf.equal(y_true, 1), y_pred, 1- y_pred)
    loss = -K.pow(1.-pt, gamma) * K.log(pt)
    loss = alpha * loss
    return tf.reduce_sum(loss, axis = -1)

In [24]:
!python train.py configs/task23.py

python: can't open file 'train.py': [Errno 2] No such file or directory


In [4]:
import torch
my_list = [torch.randn(3, 5), torch.randn(3, 5)]
result = torch.stack(my_list, dim=0).sum(dim=0)
print(result.shape) #torch.Size([5])

torch.Size([3, 5])


In [5]:
result

tensor([[ 2.3585,  0.6455, -1.1953, -1.7481,  2.4313],
        [ 0.6555, -0.7144,  2.0347, -1.1451,  1.3274],
        [-1.2196,  0.5027,  0.4278, -0.0919,  0.1193]])

In [1]:
import torchvision
m=torchvision.models.resnet34(pretrained=True)

In [2]:
m

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  